In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")


In [4]:
import pandas as pd

X_train = pd.read_csv('data/processed/X_train_res.csv')
X_test = pd.read_csv('data/processed/X_test.csv')
y_train = pd.read_csv('data/processed/y_train_res.csv')
y_test = pd.read_csv('data/processed/y_test.csv')

# Shift labels from 1,2,3 → 0,1,2
y_train = y_train - 1
y_test = y_test - 1

# Check shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Inspect first few rows
display(X_train.head())
display(y_train.head())


X_train shape: (3978, 21)
X_test shape: (426, 21)
y_train shape: (3978, 1)
y_test shape: (426, 1)


,LB,AC.1,FM.1,UC.1,DL.1,DS.1,DP.1,ASTV,MSTV,ALTV,...,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency
0,-1.032333,-0.834435,-0.206244,0.896032,0.047600,-0.059461,2.986443,-1.195728,3.331247,-0.535843,...,1.939469,-1.474154,1.782380,1.320689,1.032381,-0.687493,-1.887059,-1.726861,3.316840,-0.526713
1,0.192158,-0.575829,-0.164756,-1.484534,-0.630753,-0.059461,-0.272713,0.652431,-0.828669,0.277039,...,0.626853,-0.794055,0.055794,0.308022,-0.470307,0.101947,0.353185,0.209634,-0.575860,1.113126
2,0.294199,-0.834435,-0.206244,0.896032,0.725953,-0.059461,-0.272713,-0.675933,-0.266518,-0.535843,...,0.395215,-0.828060,-0.501169,-0.367089,-0.470307,-0.080231,-0.094864,-0.067008,-0.169665,1.113126
3,1.722772,-0.834435,-0.206244,-1.144453,-0.630753,-0.059461,-0.272713,1.980795,-1.053530,0.385423,...,-0.685763,1.246243,0.557061,-0.704645,-0.470307,0.891388,0.993254,0.901239,-0.609710,-0.526713
4,0.600322,0.717204,-0.164756,1.916275,1.404306,-0.059461,-0.272713,0.421411,0.295633,-0.535843,...,0.652590,-0.590025,0.445668,-0.029534,1.032381,1.437923,0.289178,0.486276,1.996707,1.113126


,NSP
0,2.0
1,0.0
2,0.0
3,2.0
4,0.0


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import numpy as np

kernels = ['linear', 'rbf', 'poly']

param_grids = {
    'linear': {'C':[0.1, 1, 10, 100]},
    'rbf': {'C':[0.1, 1, 10], 'gamma':['scale','auto']},
    'poly': {'C':[0.1, 1, 10], 'degree':[2, 3], 'gamma':['scale','auto']}
}


In [8]:
best_models = {}  # Initialize dictionary to store best models

for kernel in kernels:
    print(f"Training SVM with {kernel.upper()} kernel...")

    svm = SVC(kernel=kernel, probability=True, random_state=42)
    grid = GridSearchCV(svm, param_grid=param_grids[kernel], scoring='accuracy', cv=5, n_jobs=-1)
    grid.fit(X_train_scaled, y_train)

    best_svm = grid.best_estimator_
    best_models[kernel] = best_svm

    y_pred = best_svm.predict(X_test_scaled)
    report = classification_report(y_test, y_pred, output_dict=True)

    print(f"\nSVM with {kernel.upper()} Kernel - PES2UG23CS377\n")

    for cls in sorted(np.unique(y_test)):
        cls_str = str(cls)
        precision = round(report[cls_str]['precision'], 2)
        recall = round(report[cls_str]['recall'], 2)
        f1 = round(report[cls_str]['f1-score'], 2)
        support = int(report[cls_str]['support'])
        print(f"{cls_str:6} Precision: {precision}  Recall: {recall}  F1-score: {f1}  Support: {support}")

    accuracy = round(report['accuracy'], 2)
    total = len(y_test)
    print(f"\nAccuracy: {accuracy}  Total: {total}")
    macro = report['macro avg']
    print(f"Macro avg - Precision: {round(macro['precision'],2)}  Recall: {round(macro['recall'],2)}  F1-score: {round(macro['f1-score'],2)}")
    weighted = report['weighted avg']
    print(f"Weighted avg - Precision: {round(weighted['precision'],2)}  Recall: {round(weighted['recall'],2)}  F1-score: {round(weighted['f1-score'],2)}")

    print("-"*50 + "\n")


Training SVM with LINEAR kernel...


D:\Fetal-Health-Classification\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



SVM with LINEAR Kernel - PES2UG23CS377

0.0    Precision: 0.99  Recall: 0.91  F1-score: 0.95  Support: 332
1.0    Precision: 0.6  Recall: 0.86  F1-score: 0.71  Support: 59
2.0    Precision: 0.81  Recall: 0.83  F1-score: 0.82  Support: 35

Accuracy: 0.89  Total: 426
Macro avg - Precision: 0.8  Recall: 0.87  F1-score: 0.82
Weighted avg - Precision: 0.92  Recall: 0.89  F1-score: 0.9
--------------------------------------------------

Training SVM with RBF kernel...


D:\Fetal-Health-Classification\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



SVM with RBF Kernel - PES2UG23CS377

0.0    Precision: 0.97  Recall: 0.94  F1-score: 0.95  Support: 332
1.0    Precision: 0.68  Recall: 0.81  F1-score: 0.74  Support: 59
2.0    Precision: 0.91  Recall: 0.86  F1-score: 0.88  Support: 35

Accuracy: 0.91  Total: 426
Macro avg - Precision: 0.85  Recall: 0.87  F1-score: 0.86
Weighted avg - Precision: 0.92  Recall: 0.91  F1-score: 0.92
--------------------------------------------------

Training SVM with POLY kernel...


D:\Fetal-Health-Classification\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



SVM with POLY Kernel - PES2UG23CS377

0.0    Precision: 0.97  Recall: 0.93  F1-score: 0.95  Support: 332
1.0    Precision: 0.66  Recall: 0.85  F1-score: 0.74  Support: 59
2.0    Precision: 0.93  Recall: 0.8  F1-score: 0.86  Support: 35

Accuracy: 0.91  Total: 426
Macro avg - Precision: 0.85  Recall: 0.86  F1-score: 0.85
Weighted avg - Precision: 0.92  Recall: 0.91  F1-score: 0.91
--------------------------------------------------



In [9]:
import os
import joblib

# Create models folder if not already present
os.makedirs("models", exist_ok=True)

# Save the best SVM model
joblib.dump(best_svm, "models/svm_best.joblib")

print("Saved best SVM model to models/svm_best.joblib")


Saved best SVM model to models/svm_best.joblib
